# Möbius‑Digit‑Thread Scanner (v6)

Extended search:

* **Parameter sweep**
  * φ_scale centred on 0.003 944 ± 1 × 10⁻⁶ (step 2 × 10⁻⁷, 11 pts)
  * tiny exponents *k* = 4 and 4.5
  * seed bits = 64, 128
* **One shifted‑breath** evaluation.
* **Digit thread detection**
  * Substrings ≥ 4 digits that match π, φ, e, γ, δ, √2.
  * Detect monotone runs “1234”, “2345”, … and repeated runs “4444”, “5555”.
* **Power‑of‑3 proximity**
  * Flag if *x* or *x⁻¹* within 0.5 % of {9, 27, 81, 127, 729}.
* **Binary‑word arithmetic**
  * Compute Δ = |x − 0.FibBits| for 64‑ and 128‑bit Fibonacci fractions; flag if Δ < 10⁻⁵.

Outputs top 30 rows sorted by `rel_err` plus thread / proximity columns.

In [1]:
import mpmath as mp, pandas as pd, itertools, re
mp.mp.dps = 250
ALPHA_INV = mp.mpf('137.035999084'); ALPHA = 1/ALPHA_INV
PHI = (1+mp.sqrt(5))/2

# Constants digit banks
def digits_of(num, n=600):
    s=str(num).replace('.','')[1:]  # remove leading integer part
    return (s + '0'*n)[:n]

CONST_BANK = {
    'pi'   : digits_of(mp.pi),
    'phi'  : digits_of(PHI),
    'e'    : digits_of(mp.e),
    'gamma': digits_of(mp.euler),
    'delta': digits_of(mp.mpf('4.669201609')),
    'sqrt2': digits_of(mp.sqrt(2))
}


In [2]:
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))


In [3]:
def breath_shift(x,tiny,phi_scale):
    x = phi_scale*PHI / x
    x = x - tiny
    x = 1/x / 9
    x = 1/x
    x = x / 9
    return x

In [4]:
# util funcs
def const_threads(xdigits):
    for const,bank in CONST_BANK.items():
        for L in range(8,3,-1):
            for i in range(len(xdigits)-L+1):
                sub = xdigits[i:i+L]
                if sub in bank:
                    return f'{const}:{sub}'
    return '-'

def pattern_threads(xdigits):
    # ascending sequences
    asc_patterns=[ ''.join(str(int(d)+i) for i in range(4)) for d in '0123456']
    rep_patterns=[ d*4 for d in '0123456789']
    for pat in asc_patterns+rep_patterns:
        if pat in xdigits: return pat
    return '-'

powers=[9,27,81,127,729]


## Run sweep

In [5]:
rows=[]
phi_centre=0.003944
phi_vals=[phi_centre+i*2e-7 for i in range(-5,6)]  # ±1e-6
for bits in (64,128):
    fibx=fib_fraction(bits)
    for k_tiny in (4,4.5):
        tiny=ALPHA_INV/(10**k_tiny)
        for phi in phi_vals:
            x=breath_shift(fibx+tiny,tiny,phi)
            rel=float(abs((x-ALPHA)/ALPHA))
            xdig=str(x)[2:302]
            cthread=const_threads(xdig)
            pthread=pattern_threads(xdig)
            pow_hit=any(abs(float(x)-p)/p<0.005 or abs(float(1/x)-p)/p<0.005 for p in powers)
            delta=float(abs(x-mp.mpf(str(fibx))))
            fib_hit=delta<1e-5
            rows.append(dict(bits=bits,k=k_tiny,phi_scale=phi,rel_err=rel,
                             const_thread=cthread,pat_thread=pthread,
                             pow3=pow_hit,fib_near=fib_hit))
df=pd.DataFrame(rows)
df.sort_values('rel_err').head(30)


,bits,k,phi_scale,rel_err,const_thread,pat_thread,pow3,fib_near
7,64,4.0,0.003944,0.000009,pi:38196,-,False,False
29,128,4.0,0.003944,0.000009,pi:35940,-,False,False
30,128,4.0,0.003945,0.000137,pi:42117,-,False,False
8,64,4.0,0.003945,0.000137,pi:986280,-,False,False
6,64,4.0,0.003944,0.000155,pi:22317,-,False,False
28,128,4.0,0.003944,0.000155,pi:64062,-,False,False
31,128,4.0,0.003945,0.000283,pi:5940,4444,False,False
9,64,4.0,0.003945,0.000283,pi:59265,-,False,False
5,64,4.0,0.003944,0.000301,pi:4626,-,False,False
27,128,4.0,0.003944,0.000301,pi:98214,2222,False,False
